# Public Key Cipher

The encryption key being the same as the decryption key leads to a tricky problem: how do you share encrypted messages with someone you’ve never talked to before? Any eavesdroppers would be able to intercept an encryption key you send just as easily as they could intercept the encrypted messages.

Public key cryptography solves this encryption problem by using two keys, one for encryption and one for decryption, and is an example of an asymmetric cipher. Ciphers that use the same key for encryption and decryption, are symmetric ciphers.

It’s important to know that a message encrypted using the encryption key (public key) can only be decrypted using the decryption key (private key). So even if someone obtains the encryption key, they won’t be able to read the original message because the encryption key can’t decrypt the message.

The encryption key is called the public key because it can be shared with the entire world. In contrast, the private key, or the decryption key, must be kept secret.

The particular public key cipher we’ll implement is based on the RSA cipher, which was invented in 1977 and is named using the initials of the last names of its inventors: Ron Rivest, Adi Shamir, and Leonard Adleman.

The RSA cipher uses large prime numbers hundreds of digits long in its algorithm. The public key algorithm creates two random prime numbers and then uses complicated math to create the public and private keys.

# Steps for Generating Public and Private Keys
Each key in the public key scheme is made of two numbers. The public key will be the two numbers n and e. The private key will be the two numbers n and d.

The three steps to create these numbers are as follows:

* Create two random, distinct, very large prime numbers: p and q. Multiply these two numbers to get a number called n.
* Create a random number, called e, which is relatively prime to (p – 1) × (q – 1).
* Calculate the modular inverse of e, which we’ll call d.

Notice that n is used in both keys. The d number must be kept secret because it can decrypt messages. Now you’re ready to write a program that generates these keys.

In [1]:
from cracking_codes.utils import primenum, cryptomath
import os, sys, random, math

In [2]:
def generateKey(keySize):
    p = 0
    q = 0
    print('Generating p and q prime...')
    while p == q:
        p = primenum.generateLargePrime(keySize)
        q = primenum.generateLargePrime(keySize)
    n = p * q

    print('Generating e that is relatively prime to (p-1)*(q-1)...')
    while True:
        e = random.randrange(2 ** (keySize - 1), 2 ** (keySize))
        if cryptomath.gcd(e, (p - 1) * (q - 1)) == 1:
            break

    print('Calculating d that is mod inverse of e...')
    d = cryptomath.findModInverse(e, (p - 1) * (q - 1))

    publicKey = (n, e)
    privateKey = (n, d)

    print('Public key:', publicKey)
    print('Private key:', privateKey)

    return (publicKey, privateKey)


def makeKeyFiles(name, keySize):
    if os.path.exists(f'{name}_pubkey.txt') or os.path.exists(f'{name}_privkey.txt'):
        sys.exit('WARNING: The files already exists! Use a different name or delete these files and rerun this program.')

    publicKey, privateKey = generateKey(keySize)

    print()
    print(f'The public key is a {len(str(publicKey[0]))} and a {len(str(publicKey[1]))} digit number.')
    print(f'Writing public key to file {name}_pubkey.txt...')
    with open(f'{name}_pubkey.txt', 'w') as f:
        f.write(f'{keySize},{publicKey[0]},{publicKey[1]}')

    print()
    print(f'The private key is a {len(str(publicKey[0]))} and a {len(str(publicKey[1]))} digit number.')
    print(f'Writing private key to file {name}_privkey.txt...')
    with open(f'{name}_privkey.txt', 'w') as f:
        f.write(f'{keySize},{privateKey[0]},{privateKey[1]}')

In [3]:
makeKeyFiles('ritvik19', 1024)

Generating p and q prime...
Generating e that is relatively prime to (p-1)*(q-1)...
Calculating d that is mod inverse of e...
Public key: (17183626983322501017002559324039641591796758695542141994267492663265934163865332859404422662290592081979399963976998947961354950460590731968368901907711047564746246388447401964272463061464867584104361167089504185704419455151426578004954597621077268738778509572561447572705220032437024911241988050632945914303410811433986544530791797585505793583175716422959461509125773722275998968318599513707824468556391939884950212850638238519515688084009769940581452507866345450473774112211566807357919079098471472114216663854842807641437657226967064278070229884625169509718353105926472476329325557970007686430837558101598601470761, 10945236289110865848058613989395480783655700157559481608939981644799846614886215137931511588624555159008103858446079540305499260479557829507727457609396822186508103833909922858805260381524770000444915761948624848835758333364903712901974620885

# How the Public Key Cipher Works
Similar to the other ciphers, the public key cipher converts characters into numbers and then performs math on those numbers to encrypt them. What sets the public key cipher apart from other ciphers is that it converts multiple characters into one integer called a block and then encrypts one block at a time.

The reason the public key cipher needs to work on a block that represents multiple characters is that if we used the public key encryption algorithm on a single character, the same plaintext characters would always encrypt to the same ciphertext characters. Therefore, the public key cipher would be no different from a simple substitution cipher with fancy mathematics.

In [4]:
SYMBOLS = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz1234567890 !?.'

In [5]:
def getBlocksFromText(message, blockSize):
    for character in message:
        if character not in SYMBOLS:
            print(f'ERROR: The symbol set does not have the character {character}')
            sys.exit()
    blockInts = []
    for blockStart in range(0, len(message), blockSize):
        blockInt = 0
        for i in range(blockStart, min(blockStart + blockSize, len(message))):
            blockInt += (SYMBOLS.index(message[i])) * (len(SYMBOLS) ** (i % blockSize))
        blockInts.append(blockInt)
    return blockInts


def getTextFromBlocks(blockInts, messageLength, blockSize):
    message = []
    for blockInt in blockInts:
        blockMessage = []
        for i in range(blockSize - 1, -1, -1):
            if len(message) + i < messageLength:
                charIndex = blockInt // (len(SYMBOLS) ** i)
                blockInt = blockInt % (len(SYMBOLS) ** i)
                blockMessage.insert(0, SYMBOLS[charIndex])
        message.extend(blockMessage)
    return ''.join(message)


def encryptMessage(message, key, blockSize):
    encryptedBlocks = []
    n, e = key

    for block in getBlocksFromText(message, blockSize):
        encryptedBlocks.append(pow(block, e, n))
    return encryptedBlocks


def decryptMessage(encryptedBlocks, messageLength, key, blockSize):
    decryptedBlocks = []
    n, d = key
    for block in encryptedBlocks:
        decryptedBlocks.append(pow(block, d, n))
    return getTextFromBlocks(decryptedBlocks, messageLength, blockSize)


def readKeyFile(keyFilename):
    with open(keyFilename) as f:
        content = f.read()
    keySize, n, EorD = content.split(',')
    return (int(keySize), int(n), int(EorD))


def encryptAndWriteToFile(messageFilename, keyFilename, message, blockSize=None):

    keySize, n, e = readKeyFile(keyFilename)
    if blockSize == None:
        blockSize = int(math.log(2 ** keySize, len(SYMBOLS)))
    if not (math.log(2 ** keySize, len(SYMBOLS)) >= blockSize):
        sys.exit('ERROR: Block size is too large for the key and symbol set size.')
    
    encryptedBlocks = encryptMessage(message, (n, e), blockSize)

    for i in range(len(encryptedBlocks)):
        encryptedBlocks[i] = str(encryptedBlocks[i])
    encryptedContent = ','.join(encryptedBlocks)

    encryptedContent = f'{len(message)}_{blockSize}_{encryptedContent}'
    with open(messageFilename, 'w') as f:
        f.write(encryptedContent)
    return encryptedContent


def readFromFileAndDecrypt(messageFilename, keyFilename):
    keySize, n, d = readKeyFile(keyFilename)

    with open(messageFilename) as f:
        content = f.read()
    messageLength, blockSize, encryptedMessage = content.split('_')
    messageLength = int(messageLength)
    blockSize = int(blockSize)

    if not (math.log(2 ** keySize, len(SYMBOLS)) >= blockSize):
        sys.exit('ERROR: Block size is too large for the key and symbol set size.')

    encryptedBlocks = []
    for block in encryptedMessage.split(','):
        encryptedBlocks.append(int(block))

    return decryptMessage(encryptedBlocks, messageLength, (n, d),blockSize)

In [6]:
message = """Journalists belong in the gutter because that is where the ruling classes throw their guilty secrets. Gerald Priestland. The Founding Fathers gave the free press the protection it must have to bare the secrets of government and inform the people. Hugo Black."""

filename = 'encrypted_file.txt'

In [7]:
pubKeyFilename = 'ritvik19_pubkey.txt'
print('Encrypting and writing to file...')
encryptedText = encryptAndWriteToFile(filename, pubKeyFilename, message)

print('Encrypted text:')
print(encryptedText)

Encrypting and writing to file...
Encrypted text:
258_169_7089811149173130184902652642424232032553743380939344390616651100534222437748524339109708411109722863069767032980440712678046358339504859033337072931739432989525612236430763138860952047268363140776397359278039221070013433203310467744124256226860162779790602877830337547445051990309891807118520347825543821202377555182287605460302538947295634248773533415892048617059249365487430754547896784705779892800032069090807064681457628563324551345321073074480683082431097853521418718996941642486584959502374812337486741894939905788795309116141940041442071722688476694795604290943124741000037011974135524042808627294027293348050,1230210785573209114135071661909699465588725034840462615330609599633312179199026927744504992962168103545900548173339408623238596012003059690967466212053588526553660196098568357506539997958892765330257332348315766674010179473368692464868617546229390336237434047426139923892041186462963458617036557736011359523292537966161808712

In [8]:
privKeyFilename = 'ritvik19_privkey.txt'
print('Reading from file and decrypting...')
decryptedText = readFromFileAndDecrypt(filename, privKeyFilename)

print('Decrypted text:')
print(decryptedText)

Reading from file and decrypting...
Decrypted text:
Journalists belong in the gutter because that is where the ruling classes throw their guilty secrets. Gerald Priestland. The Founding Fathers gave the free press the protection it must have to bare the secrets of government and inform the people. Hugo Black.
